In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shutil

# Function to preprocess an image
def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Function to build the initial model
def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    predictions = tf.keras.layers.Dense(4, activation='softmax')(x)  # 4 classes including 'Other'

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to get the data
def get_data():
    data = []
    labels = []

    for label, folder in enumerate(['Butterfly']):
        folder_path = os.path.join('raw-img', folder)
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = Image.open(img_path)
            data.append(preprocess_image(img))
            labels.append(label)

    return np.vstack(data), np.array(labels)

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Function to retrain the model with user feedback
def retrain_model(model, feedback_images):
    data = []
    labels = []

    for label, folder in enumerate(['Butterfly']):
        folder_path = os.path.join(feedback_images, folder)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder '{folder}' does not exist in '{feedback_images}'.")
            continue
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = Image.open(img_path)
            data.append(preprocess_image(img))
            labels.append(label)

    if data and labels:
        X = np.vstack(data)
        y = np.array(labels)
        y = tf.keras.utils.to_categorical(y, num_classes=4)
        model.fit(X, y, epochs=1, batch_size=32)
    else:
        print("No feedback images found.")


In [2]:
# Training and evaluation
def main():
    model = build_model()
    X, y = get_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
    model.fit(X_train, y_train, epochs=1, batch_size=32)

    print("Model Performance:")
    evaluate_model(model, X_test, y_test)

    feedback_images_dir = 'feedback_images'
    if os.path.exists(feedback_images_dir):
        shutil.rmtree(feedback_images_dir)
    os.makedirs(feedback_images_dir, exist_ok=True)

    # Assume 'feedback_images' is the folder where users' feedback images are stored
    retrain_model(model, 'feedback_images')

    # Evaluate the model again after retraining
    print("Model Performance After Retraining:")
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    main()

def train_model():
    X, y = get_data()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)  # Change to 4 classes
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)    # Change to 4 classes

    model = build_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    evaluate_model(model, X_test, np.argmax(y_test, axis=1))
    
    # Save the trained model
    model.save('image_classifier_model.pt')

    return model

IndexError: index 7 is out of bounds for axis 1 with size 4

In [ ]:
# Train the model
trained_model = train_model()